# Available XC functionals

A number of exchange–correlation (XC) functionals are available in VeloxChem.

We will calculate the electronic energy of helium using the Hartree–Fock and second-order Møller–Plesset methods as well as Kohn–Sham density functional theory using the available XC functionals.

In [ ]:
import matplotlib.pyplot as plt
import veloxchem as vlx

In [ ]:
molecule = vlx.Molecule.read_str("He   0.000  0.000  0.000")
basis = vlx.MolecularBasis.read(molecule, "cc-pvtz")

## Hartree–Fock

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()

scf_results = scf_drv.compute(molecule, basis)

scf_energies = {}
scf_energies["HF"] = scf_drv.get_scf_energy()

## MP2

In [ ]:
mp2_drv = vlx.Mp2Driver()
mp2_results = mp2_drv.compute(molecule, basis, scf_drv.mol_orbs)

scf_energies["MP2"] = mp2_results["mp2_energy"] + scf_energies["HF"]

## DFT

In [ ]:
for xcfun in vlx.available_functionals():

    if xcfun.lower() in ['vwn_rpa', 'lyp']:
        continue
        
    scf_drv.xcfun = xcfun
    scf_drv.compute(molecule, basis)
    scf_energies[xcfun] = scf_drv.get_scf_energy()

## Plotting the energies

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

number_methods = len(scf_energies)

for i, method in enumerate(scf_energies.keys()):

    ax.plot(i, scf_energies[method], "o")

ax.set_xticks(range(number_methods), scf_energies.keys())
plt.xticks(rotation=90)

ax.set_ylim(-2.93, -2.82)
plt.ylabel("Electronic energy (a.u.)")

plt.grid(True)

plt.show()